<a href="https://colab.research.google.com/github/RodrigoGuedesDP/Computer_Vision/blob/main/cv_labo3_vgg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Computer Vision - Clasificación de expresiones faciales con VGG16
**

---



*   En este laboratorio se construira usara el modelo VGG16 pre-entrenado para construir un clasificador con estos datos y evaluar su rendimiento. Para ello se utilizará un dataset público llamado "FER-2013".

**Autores:**  

Nieto Espinoza, Brajan E.  
[brajan.nieto@utec.edu.pe](mailto:brajan.nieto@utec.edu.pe)

Guedes del Pozo,  Rodrigo J.  
[rodrigo.guedes.d@utec.edu.pe](mailto:rodrigo.guedes.d@utec.edu.pe)

<img src="https://pregrado.utec.edu.pe/sites/default/files/logo-utec-h_0_0.svg" width="190" alt="Logo UTEC" loading="lazy" typeof="foaf:Image">      

---

# Sesión 08: Arquitectura VGG

En esta sesión estudiaremos la **arquitectura VGG**, una de las redes neuronales convolucionales más influyentes en visión por computador. La red VGG, propuesta por el *Visual Geometry Group* de la Universidad de Oxford, se caracteriza por utilizar filtros de **3×3** de forma uniforme y apilar muchas capas para capturar patrones de las imágenes.

Exploraremos cómo cargar el modelo VGG16 utilizando `torchvision`, analizaremos su número de parámetros y construiremos la arquitectura **desde cero** con PyTorch para comprender su estructura interna. También aprenderemos a usar un modelo VGG16 preentrenado en ImageNet para clasificar imágenes nuevas.


## Cargando la arquitectura VGG desde `torchvision`

Para comenzar, cargaremos la clase `VGG16` desde `torchvision.models`. Al especificar `weights=None` obtendremos la arquitectura sin pesos preentrenados. A continuación mostramos un resumen del modelo (capas, tamaños de salida y número de parámetros de cada capa) utilizando la utilidad `torchinfo.summary`.

> **Nota:** Para usar `summary` se necesita instalar el paquete `torchinfo` (si no está instalado en tu entorno). Google Colab ya incluye PyTorch y Torchvision, pero puedes instalar dependencias adicionales mediante `pip`.


In [ ]:
# Instalamos la biblioteca torchinfo en caso de que no esté disponible
!pip install -q torchinfo

import torch
import torchvision.models as models
from torchinfo import summary

# Cargar la arquitectura VGG16 sin pesos preentrenados
vgg16_arch = models.vgg16(weights=None)

# Mostramos un resumen del modelo. La entrada es una imagen RGB de 224×224
summary(vgg16_arch, input_size=(1, 3, 224, 224))


Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [1, 1000]                 --
├─Sequential: 1-1                        [1, 512, 7, 7]            --
│    └─Conv2d: 2-1                       [1, 64, 224, 224]         1,792
│    └─ReLU: 2-2                         [1, 64, 224, 224]         --
│    └─Conv2d: 2-3                       [1, 64, 224, 224]         36,928
│    └─ReLU: 2-4                         [1, 64, 224, 224]         --
│    └─MaxPool2d: 2-5                    [1, 64, 112, 112]         --
│    └─Conv2d: 2-6                       [1, 128, 112, 112]        73,856
│    └─ReLU: 2-7                         [1, 128, 112, 112]        --
│    └─Conv2d: 2-8                       [1, 128, 112, 112]        147,584
│    └─ReLU: 2-9                         [1, 128, 112, 112]        --
│    └─MaxPool2d: 2-10                   [1, 128, 56, 56]          --
│    └─Conv2d: 2-11                      [1, 256, 56, 56]          29

## Número total de parámetros de VGG16

El número de parámetros de una red es una medida de su complejidad y capacidad de representación. Podemos sumar el número de elementos de cada tensor de parámetros utilizando `p.numel()` para obtener el total de parámetros entrenables.  

La documentación oficial de PyTorch indica que la variante VGG16 preentrenada contiene **138 357 544** parámetros. Comprobemos este valor mediante código:


In [ ]:
# Calcular el número total de parámetros de la arquitectura VGG16
# Sumamos todos los elementos de cada tensor de parámetros
total_params = sum(p.numel() for p in vgg16_arch.parameters())
print(f"Total de parámetros (sin pesos preentrenados): {total_params} ≈ {total_params/1e6:.2f} millones")


Total de parámetros (sin pesos preentrenados): 138357544 ≈ 138.36 millones


## VGG16 desde cero con PyTorch

Para comprender mejor la estructura de VGG16 implementaremos la red manualmente. VGG16 sigue un patrón simple: bloques de convoluciones 3×3 repetidas, seguidos de max pooling 2×2 para reducir la resolución. La configuración típica de VGG16 es la siguiente:

- **Bloque 1:** Conv 64 → Conv 64 → MaxPool 2×2  
- **Bloque 2:** Conv 128 → Conv 128 → MaxPool 2×2  
- **Bloque 3:** Conv 256 → Conv 256 → Conv 256 → MaxPool 2×2  
- **Bloque 4:** Conv 512 → Conv 512 → Conv 512 → MaxPool 2×2  
- **Bloque 5:** Conv 512 → Conv 512 → Conv 512 → MaxPool 2×2  

Después de los bloques convolucionales, la salida se aplana y pasa a través de tres capas completamente conectadas (`Linear`) con 4096, 4096 y `num_classes` neuronas, respectivamente. Esta estructura, junto con el uso constante de filtros 3×3, permite captar patrones detallados sin aumentar excesivamente el número de parámetros.


In [ ]:
import torch
from torch import nn
from torchinfo import summary

class VGG(nn.Module):
    """
    Implementación de la arquitectura VGG16.
    Args:
        num_classes (int): número de clases de salida (por defecto 1000).
    """
    def __init__(self, num_classes: int = 1000) -> None:
        super().__init__()
        # Funciones de activación, pooling y dropout reutilizables
        self.relu    = nn.ReLU(inplace=True)
        self.pool    = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout()

        # Bloque 1
        self.conv1_1 = nn.Conv2d(3,   64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(64,  64, kernel_size=3, padding=1)

        # Bloque 2
        self.conv2_1 = nn.Conv2d(64,  128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)

        # Bloque 3
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)

        # Bloque 4
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

        # Bloque 5
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

        # Agrupación y clasificador
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.fc1 = nn.Linear(512 * 7 * 7, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, num_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Bloque 1
        x = self.relu(self.conv1_1(x))
        x = self.relu(self.conv1_2(x))
        x = self.pool(x)

        # Bloque 2
        x = self.relu(self.conv2_1(x))
        x = self.relu(self.conv2_2(x))
        x = self.pool(x)

        # Bloque 3
        x = self.relu(self.conv3_1(x))
        x = self.relu(self.conv3_2(x))
        x = self.relu(self.conv3_3(x))
        x = self.pool(x)

        # Bloque 4
        x = self.relu(self.conv4_1(x))
        x = self.relu(self.conv4_2(x))
        x = self.relu(self.conv4_3(x))
        x = self.pool(x)

        # Bloque 5
        x = self.relu(self.conv5_1(x))
        x = self.relu(self.conv5_2(x))
        x = self.relu(self.conv5_3(x))
        x = self.pool(x)

        # Agrupación y capas densas
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

# Creamos una instancia del modelo VGG16
vgg16_custom = VGG(num_classes=1000)

# Resumen de la red personalizada para verificar coincidencia con VGG16
summary(vgg16_custom, input_size=(1, 3, 224, 224))


Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [1, 1000]                 --
├─Conv2d: 1-1                            [1, 64, 224, 224]         1,792
├─ReLU: 1-2                              [1, 64, 224, 224]         --
├─Conv2d: 1-3                            [1, 64, 224, 224]         36,928
├─ReLU: 1-4                              [1, 64, 224, 224]         --
├─MaxPool2d: 1-5                         [1, 64, 112, 112]         --
├─Conv2d: 1-6                            [1, 128, 112, 112]        73,856
├─ReLU: 1-7                              [1, 128, 112, 112]        --
├─Conv2d: 1-8                            [1, 128, 112, 112]        147,584
├─ReLU: 1-9                              [1, 128, 112, 112]        --
├─MaxPool2d: 1-10                        [1, 128, 56, 56]          --
├─Conv2d: 1-11                           [1, 256, 56, 56]          295,168
├─ReLU: 1-12                             [1, 256, 56, 56]       

En el resumen anterior, la arquitectura de VGG16 implementada manualmente coincide con la versión proporcionada por `torchvision`. Observa que el número total de parámetros también se aproxima a 138 millones, igual que la versión preentrenada.


## Cargando VGG16 preentrenado en ImageNet

`torchvision` permite cargar modelos VGG16 con pesos preentrenados en el conjunto de datos **ImageNet**. Estos modelos han aprendido representaciones generales de objetos y pueden utilizarse para clasificación o como extractores de características en otras tareas.  

Al cargar el modelo con `weights='IMAGENET1K_V1'` obtendremos automáticamente los pesos entrenados. A continuación mostraremos un resumen de la arquitectura preentrenada.


In [ ]:
import torchvision.models as models
from torchinfo import summary

# Cargar VGG16 con pesos preentrenados en ImageNet
vgg16_pretrained = models.vgg16(weights='IMAGENET1K_V1')

# Mostrar resumen del modelo preentrenado
summary(vgg16_pretrained, input_size=(1, 3, 224, 224))


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


100%|██████████| 528M/528M [00:07<00:00, 77.2MB/s]


Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [1, 1000]                 --
├─Sequential: 1-1                        [1, 512, 7, 7]            --
│    └─Conv2d: 2-1                       [1, 64, 224, 224]         1,792
│    └─ReLU: 2-2                         [1, 64, 224, 224]         --
│    └─Conv2d: 2-3                       [1, 64, 224, 224]         36,928
│    └─ReLU: 2-4                         [1, 64, 224, 224]         --
│    └─MaxPool2d: 2-5                    [1, 64, 112, 112]         --
│    └─Conv2d: 2-6                       [1, 128, 112, 112]        73,856
│    └─ReLU: 2-7                         [1, 128, 112, 112]        --
│    └─Conv2d: 2-8                       [1, 128, 112, 112]        147,584
│    └─ReLU: 2-9                         [1, 128, 112, 112]        --
│    └─MaxPool2d: 2-10                   [1, 128, 56, 56]          --
│    └─Conv2d: 2-11                      [1, 256, 56, 56]          29

## Uso del VGG16 preentrenado para clasificar imágenes

Una de las principales aplicaciones de VGG16 es clasificar imágenes en las 1000 categorías de ImageNet. Para realizar inferencia:

1. **Preprocesamiento:** La imagen debe redimensionarse a 256 × 256, realizar un recorte central de 224 × 224 y normalizarse con las medias y desviaciones estándar recomendadas. Estos pasos pueden obtenerse directamente de `VGG16_Weights.IMAGENET1K_V1.transforms()`.
2. **Inferencia:** Se pasa la imagen a través de la red en modo evaluación (`model.eval()`), se obtiene la salida y se aplica `argmax` para seleccionar la clase de mayor probabilidad.
3. **Decodificación:** La lista de categorías está disponible en `weights.meta['categories']` y permite traducir el índice de la clase al nombre legible.

A continuación se muestra una función que encapsula todo el proceso. La función recibe la ruta de la imagen, realiza el preprocesamiento, ejecuta la inferencia y devuelve la clase predicha.


In [ ]:
from PIL import Image
import torch
import torchvision.transforms as transforms
from torchvision.models import vgg16, VGG16_Weights

# Función para cargar el modelo preentrenado y las transformaciones
def load_pretrained_vgg():
    weights = VGG16_Weights.IMAGENET1K_V1
    model = vgg16(weights=weights)
    model.eval()  # establecer en modo evaluación
    preprocess = weights.transforms()  # incluye resize, center crop y normalización
    categories = weights.meta['categories']
    return model, preprocess, categories

# Función para clasificar una imagen dada la ruta
def classify_image(image_path: str):
    """
    Clasifica una imagen usando VGG16 preentrenado en ImageNet.

    Args:
        image_path (str): ruta al archivo de imagen (JPEG/PNG).

    Returns:
        str: nombre de la clase predicha.
    """
    model, preprocess, categories = load_pretrained_vgg()
    # Abrir imagen y convertir a RGB
    image = Image.open(image_path).convert('RGB')
    # Aplicar transformaciones y crear batch (agregar dimensión batch)
    input_tensor = preprocess(image).unsqueeze(0)
    with torch.no_grad():
        outputs = model(input_tensor)
    predicted_idx = outputs.argmax(dim=1).item()
    predicted_class = categories[predicted_idx]
    return predicted_class

In [ ]:
image_path = 'imagen.jpg'
print('Predicción:', classify_image(image_path))


Predicción: lorikeet
